<center>
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates how to copy an AMI from one region to multiple AWS regions using unSkript actions.</b>
</div>

<br>
</center>
<center><h2>Copy AMI To All Given AWS Regions</h2></center>

# Steps Overview
   1)[Get All AWS Regions](#1)</br>
    
   2)[Copy AMI To Other AWS Regions](#2)

### <a id="1"> Get All AWS Regions</a>
Here in step-1 we will collect all the regions from the AWS by using a command `aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text`. This action only executes when the user doesn't provide the destination regions.

>Input parameters: `aws_command`

>Ouput variable: `destination_regions`

In [12]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint
from beartype import beartype


@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> list:
    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_command": "\\"aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not destination_regions",
    "condition_result": true
    }''')
task.configure(outputName="destination_regions")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

### <a id="2"> Copy AMI To Other AWS Regions</a>
In this action, we will provide all the necessary inputs to copy the AMI from one region to another. If the user did not provide the destination region, we can collect all the AWS regions from step 1 and copy the AMI to all the regions; if the user provided the destination region, this action copies the AMI to only the given region by using a command `aws ec2 copy-image --source-image-id <ami-id> --source-region <source-region> --region <dest-region> --name <ami-name>`.

>Input parameters: `aws_command`, `AMI_Id`, `SourceRegion`,`Name`, `DestinationRegion`

>Ouput variable: `command_output`

In [82]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#
from pydantic import BaseModel, Field
import pprint
from beartype import beartype


@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str, AMI_Id: str, SourceRegion: str,
                            DestinationRegion: str, Name: str) -> str:
    aws_command = aws_command.format(AMI_Id=AMI_Id, SourceRegion=SourceRegion,
                                     DestinationRegion=DestinationRegion, Name=Name)
    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "AMI_Id": "ami_id",
    "DestinationRegion": "iter_item",
    "Name": "ami_name",
    "SourceRegion": "source_region",
    "aws_command": "\\"aws ec2 copy-image --source-image-id {AMI_Id} --source-region {SourceRegion} --region {DestinationRegion} --name {Name}\\""
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "destination_regions",
    "iter_parameter": "DestinationRegion"
    }''')
task.configure(outputName="command_output")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

   ### Conclusion
   In this Runbook, we demonstrated the use of unSkript's AWS actions to perform copy AMI from one region to multiple AWS regions.
   To view the full platform capabilities of unSkript please visit https://us.app.unskript.io